 <font size="5" color="black">一.项目背景介绍：</font>近年来，随着人们对更美好生活的向往，国家大力推动垃圾分类活动，以大城市作为试点，逐步推广，然而，不管是大城市还是小城市，人们普遍对垃圾分类有关知识了解甚少，尤其是在试点城市，很多市民错放垃圾，为向大家普及有关知识，通过计算机视觉的方法，在实地进行垃圾分类之前可以正确知道该如何分类，可以减少垃圾集中处理点的工作人员的工作量，提高效率，为建设美好中国做贡献。

 <font size="5" color="black">二.数据介绍：</font>使用paddle上的数据集，有关垃圾分类的，数量69003

 <font size="5" color="black">paddlex准备：</font>

In [1]:
#!pip install 'paddlex<=2.2-rc'
!pip install paddlex

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.6 MB 6.4 MB/s            
     |████████████████████████████████| 2.0 MB 18.9 MB/s            
     |████████████████████████████████| 6.8 MB 3.4 MB/s            
     |████████████████████████████████| 2.7 MB 5.0 MB/s            
     |████████████████████████████████| 310 kB 6.3 MB/s            
     |████████████████████████████████| 106 kB 6.0 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 151 kB 4.4 MB/s            
     |████████████████████████████████| 1.5 MB 9.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 50 kB 205 kB/s             
     |████████████████████████████████| 296 kB 775 kB/s            
     |████████████████████████████████| 98 kB 188 kB/s             
     |████████

In [2]:
!pip list

Package                Version
---------------------- ---------------
absl-py                0.8.1
alembic                1.7.5
anyio                  3.5.0
argcomplete            2.0.0
argon2-cffi            21.3.0
argon2-cffi-bindings   21.2.0
aspy.yaml              1.3.0
astor                  0.8.1
astroid                2.4.1
async-generator        1.10
attrs                  21.4.0
audioread              2.1.8
autopep8               1.6.0
Babel                  2.8.0
backcall               0.1.0
bce-python-sdk         0.8.53
blackhole              1.0.1
bleach                 4.1.0
cachetools             4.0.0
certifi                2019.9.11
certipy                0.1.3
cffi                   1.15.0
cfgv                   2.0.1
chardet                3.0.4
Click                  7.0
cloudpickle            1.6.0
cma                    2.7.0
colorama               0.4.4
colorlog               4.1.0
cryptography           36.0.1
cycler                 0.10.0
Cython                 

In [3]:
!unzip -oq /home/aistudio/data/data101031/Garbages.zip -d work/ 

In [4]:
!paddlex --split_dataset --format ImageNet --dataset_dir work --val_value 0.2 --test_value 0.1 

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
[02-24 13:15:35 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format ImageNet --dataset_dir work --val_value 0.2 --test_value 0.1
[02-24 13:15:35 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:1

In [5]:
import matplotlib
matplotlib.use('Agg')
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import paddlex as pdx

[02-24 13:16:38 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()


In [6]:
from paddlex import transforms as T
train_transforms = T.Compose(
    [T.RandomCrop(crop_size=224), T.RandomHorizontalFlip(), T.Normalize()])

eval_transforms = T.Compose([
    T.ResizeByShort(short_size=256), T.CenterCrop(crop_size=224), T.Normalize()
])

In [7]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='work',
    file_list='work/train_list.txt',
    label_list='work/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.ImageNet(
    data_dir='work',
    file_list='work/val_list.txt',
    label_list='work/labels.txt',
    transforms=eval_transforms)

2022-02-24 13:17:33 [INFO]	Starting to read file list from dataset...
2022-02-24 13:17:34 [INFO]	48300 samples in file work/train_list.txt
2022-02-24 13:17:34 [INFO]	Starting to read file list from dataset...
2022-02-24 13:17:34 [INFO]	13800 samples in file work/val_list.txt


 <font size="5" color="black">三.模型介绍：</font>本文档中使用百度基于蒸馏方法得到的MobileNetV3预训练模型，模型结构与MobileNetV3一致

 <font size="5" color="black">四.模型训练：</font>

In [ ]:
num_classes = len(train_dataset.labels)
model = pdx.cls.ResNet50_vd_ssld(num_classes=len(train_dataset.labels))

model.train(
    num_epochs=5,
    train_dataset=train_dataset,
    train_batch_size=32,
    eval_dataset=eval_dataset,
    lr_decay_epochs=[4, 6, 8],
    learning_rate=0.025,
    save_dir='output/resnet50_vd_ssld',
    use_vdl=True)

2022-02-24 13:17:58 [INFO]	Downloading ResNet50_vd_ssld_pretrained.pdparams from https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/legendary_models/ResNet50_vd_ssld_pretrained.pdparams


100%|██████████| 151509/151509 [00:02<00:00, 61569.81KB/s]


2022-02-24 13:18:01 [INFO]	Loading pretrained model from output/resnet50_vd_ssld/pretrain/ResNet50_vd_ssld_pretrained.pdparams
2022-02-24 13:18:02 [WARNING]	[SKIP] Shape of pretrained params fc.weight doesn't match.(Pretrained: (2048, 1000), Actual: [2048, 5])
2022-02-24 13:18:02 [WARNING]	[SKIP] Shape of pretrained params fc.bias doesn't match.(Pretrained: (1000,), Actual: [5])
2022-02-24 13:18:02 [INFO]	There are 275/277 variables loaded into ResNet50_vd_ssld.
2022-02-24 13:21:04 [INFO]	[TRAIN] Epoch=1/5, Step=10/1509, loss=0.658074, acc1=0.812500, acc5=1.000000, lr=0.025000, time_each_step=18.18s, eta=40:14:8
2022-02-24 13:24:01 [INFO]	[TRAIN] Epoch=1/5, Step=20/1509, loss=0.498513, acc1=0.812500, acc5=1.000000, lr=0.025000, time_each_step=17.69s, eta=39:5:21
2022-02-24 13:26:59 [INFO]	[TRAIN] Epoch=1/5, Step=30/1509, loss=0.876329, acc1=0.781250, acc5=1.000000, lr=0.025000, time_each_step=17.88s, eta=39:28:17
2022-02-24 13:29:55 [INFO]	[TRAIN] Epoch=1/5, Step=40/1509, loss=0.901564

<font size="5" color="black">五.模型评估：</font>

In [ ]:
import paddlex as pdx
model = pdx.load_model('output/resnet50_vd_ssld/best_model')
image_name = 'work/evalImageSet/70.jpg'
result = model.predict(image_name)
print("Predict Result:", result)

 <font size="5" color="black">六.个人总结：</font>新手小白，大一新生，在这里学习了很多，虽然可能还没有上路，有许多还不懂，希望可以好好学习以后。

<font size="5" color="black">七.总结与升华：</font>我国经济快速增长，各项建设取得巨大成就，但也付出了巨大的资源和环境被破坏的代价，这两者之间的矛盾日趋尖锐，群众对环境污染问题反应强烈。这种状况与经济结构不合理、增长方式粗放直接相关。不加快调整经济结构，转变增长方式，资源支撑不住，环境容纳不下，社会承受不起，经济发展难以为继。只有坚持节约发展、清洁发展、安全发展，才能实现经济又好又快发展。垃圾分类是垃圾进行科学处理的前提，为垃圾的减量化、资源化、无害化处理奠定基础。普及环保与垃圾的知识，提升全社会对环卫行业的认知，减少环卫工人的工作难度，形成尊重、关心环卫工人的氛围。垃圾分类是对垃圾进行前处置的重要环节。通过分类投放、分类收集，把有用物资，如纸张、塑料、橡胶、玻璃、瓶罐、金属以及废旧家用电器等从垃圾中分离出来重新回收、利用，变废为宝。既提高垃圾资源利用水平，又可减少垃圾处置量。它是实现垃圾减量化和资源化的重要途径和手段。垃圾通过分类收集后便于对不同类垃圾进行分类处置。如对有机垃圾进行堆肥发酵处理，把有机垃圾制成农田用肥和绿化用肥，对没有回收利用价值的无机垃圾进行填埋处置，对热值较高的可燃垃圾进行焚烧处置。垃圾分类是对垃圾收集处置传统方式的改革，是对垃圾进行有效处置的一种科学管理方法。人们面对日益增长的垃圾产量和环境状况恶化的局面，如何通过垃圾分类管理，最大限度地实现垃圾资源利用，减少垃圾处置量，改善生存环境质量，是当前世界各国共同关注的迫切问题之一。

In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [0]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 